### **Direct Preference Optimization (DPO) usando Hugging Face**

Los modelos de lenguaje a gran escala (LLMs) han revolucionado el campo del procesamiento de lenguaje natural (NLP) al lograr un rendimiento excepcional en diversas tareas. Sin embargo, resulta desafiante alinear estos modelos con las preferencias humanas. Por ello, surge el método Direct Preference Optimization (DPO), que optimiza directamente los modelos basados en LLM según las preferencias de los usuarios, mejorando su alineación con las expectativas humanas. 

En este cuaderno práctico, utilizaremos la librería de refuerzo de transformers (`trl`) de Hugging Face para implementar DPO y ajustar finamente los LLM.


### **Configuración**

#### Instalación de las librerías requeridas


In [ ]:
!pip install torch
!pip install trl      # para el entrenamiento de optimización
!pip install peft     # para crear la arquitectura LoRA
!pip install matplotlib

#### **Importación de las librerías necesarias**

*Se recomienda importar todas las librerías requeridas en un mismo lugar (aquí):*

In [ ]:
import multiprocessing
import os
import requests
import tarfile
import pandas as pd
import matplotlib.pyplot as plt

import torch
from datasets import load_dataset

from peft import LoraConfig
from transformers import AutoModelForCausalLM, AutoTokenizer,TrainingArguments, GPT2Tokenizer, set_seed, GenerationConfig
from trl import DPOConfig, DPOTrainer


#### **Creación y configuración el modelo y el tokenizador**


In [ ]:
# Carga el modelo GPT-2
modelo = AutoModelForCausalLM.from_pretrained("gpt2")

# Carga un modelo de referencia
model_ref = AutoModelForCausalLM.from_pretrained("gpt2")

# Carga el tokenizador GPT-2
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Establece el token de relleno al token de fin de secuencia
tokenizer.pad_token = tokenizer.eos_token
# Establece el lado de padding a "right" para evitar problemas de desbordamiento con FP16
tokenizer.padding_side = "right"

# Deshabilita el uso de la caché durante la pasada hacia adelante del modelo
modelo.config.use_cache = False

Aquí puedes revisar la arquitectura del modelo:

In [ ]:
modelo

#### **Configuración de modelo cuantizado (Opcional)**

Si deseas un entrenamiento más eficiente en memoria y dispones de un entorno con GPU, puedes descargar el cuaderno completo, descomentar los bloques de código siguientes para crear un modelo cuantizado y continuar el entrenamiento en GPU. 

Esto se debe a que necesitarás GPUs para el paquete `bitsandbytes`.

In [ ]:
#!pip install -U bitsandbytes # este paquete es requerido para la cuantización

***Nota:***  *Puedes ejecutar el paquete instalado reiniciando el Kernel.*


In [ ]:
'''## Modelo cuantizado -- solo disponible en GPU
from transformers import BitsAndBytesConfig

# Configura los parámetros de cuantización
quantization_config = BitsAndBytesConfig(
    # Carga el modelo en formato cuantizado de 4 bits
    load_in_4bit=True,
    # Habilita la doble cuantización para una mayor precisión
    bnb_4bit_use_double_quant=True,
    # Usa cuantización no uniforme de 4 bits (nf4)
    bnb_4bit_quant_type="nf4",
    # Usa bfloat16 como tipo de dato de cómputo durante la cuantización
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Carga el modelo GPT-2 con la configuración de cuantización especificada
modelo = AutoModelForCausalLM.from_pretrained("gpt2", quantization_config=quantization_config)

# Carga un modelo de referencia con la misma configuración de cuantización
model_ref = AutoModelForCausalLM.from_pretrained("gpt2", quantization_config=quantization_config)

# Carga el tokenizador GPT-2
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Establece el token de relleno al token de fin de secuencia
tokenizer.pad_token = tokenizer.eos_token
# Establece el lado de padding a "right" para evitar problemas de desbordamiento con FP16
tokenizer.padding_side = "right"

# Deshabilita el uso de la caché durante la pasada hacia adelante del modelo
modelo.config.use_cache = False
'''

#### **Preprocesamiento del conjunto de datos**

El conjunto de datos `ultrafeedback_binarized` en Hugging Face es una colección de prompts y respuestas.


In [ ]:
# Carga el conjunto de datos desde la ubicación especificada
ds = load_dataset("BarraHome/ultrafeedback_binarized")

Este conjunto de datos incluye seis particiones (splits). 


In [ ]:
ds.keys()

Cada registro posee varias características, entre las cuales debes seleccionar tres: `"chosen"`, `"rejected"` y `"prompt"`. Esto significa que para cada prompt se proporciona una respuesta preferida y una rechazada.

In [ ]:
ds["train_prefs"][0].keys()

Puedes revisar un registro de muestra, donde verás las tres características principales: el prompt, la respuesta rechazada y la respuesta elegida.


In [ ]:
ds["train_prefs"][0]

Ahora, coloca el conjunto de datos en el formato que acepta el entrenador DPO:

| Chosen | Rejected | Prompt |
| --- | --- | --- |
 | Developing a daily habit of drawing can be challenging <br>but with consistent practice, and a few tips. | One way to develop a habit of drawing daily is <br>to allocate a specific time interval for drawing. | How can I develop a habit of drawing daily?|


In [ ]:
# Puedes reducir el volumen de datos (debido a limitaciones de recursos) seleccionando el primer 5 % de ejemplos de cada partición del conjunto de datos
for key in ds:
    #cnt = round(ds[key].__len__()*0.05)
    cnt = 50
    ds[key] = ds[key].select(range(cnt))

# Define una función para procesar los datos
def process(row):
    # elimina columnas no deseadas
    del row["prompt_id"]
    del row["messages"]
    del row["score_chosen"]
    del row["score_rejected"]
    # obtiene el texto real de la respuesta
    row["chosen"] = row["chosen"][-1]["content"]
    row["rejected"] = row["rejected"][-1]["content"]

    return row

# Aplica la función de procesamiento al conjunto de datos
ds = ds.map(
    process,
    num_proc=multiprocessing.cpu_count(),
    load_from_cache_file=False,
)

# Separa el conjunto de datos en entrenamiento y evaluación
train_dataset = ds['train_prefs']
eval_dataset = ds['test_prefs']

Vamos a revisar un registro de datos:


In [ ]:
train_dataset[0]

A continuación, definimos la configuración de LoRA para un afinamiento eficiente:


In [ ]:
# Configuración PEFT (Parameter-Efficient Fine-Tuning)
peft_config = LoraConfig(
    # Rango de las matrices de adaptación rango bajo
    r=4,
    # Módulos objetivo a los que se aplicará la adaptación de rango bajo
    target_modules=['c_proj','c_attn'],
    # Tipo de tarea para la adaptación de baja-rank
    task_type="CAUSAL_LM",
    # Factor de escala para las matrices de adaptación de rango bajo
    lora_alpha=8,
    # Probabilidad de dropout para las matrices de adaptación de rango bajo
    lora_dropout=0.1,
    # Modo de sesgo para la adaptación de rango bajo
    bias="none",
)

#### **Configuración DPO**

Primero, define los argumentos de entrenamiento:

In [ ]:
# Configuración DPO
training_args = DPOConfig(
    # Parámetro beta para la función de pérdida DPO.
    # beta es el parámetro de temperatura para la pérdida DPO, típicamente en el rango 0.1–0.5.
    beta=0.1,
    # Directorio de salida para el entrenamiento
    output_dir="dpo",
    # Número de épocas de entrenamiento
    num_train_epochs=5,
    # Tamaño de lote por dispositivo durante el entrenamiento
    per_device_train_batch_size=1,
    # Tamaño de lote por dispositivo durante la evaluación
    per_device_eval_batch_size=1,
    # Si se eliminan columnas no utilizadas del conjunto de datos
    remove_unused_columns=False,
    # Número de pasos entre registros de progreso
    logging_steps=10,
    # Número de pasos de acumulación de gradiente
    gradient_accumulation_steps=1,
    # Tasa de aprendizaje para la optimización
    learning_rate=1e-4,
    # Estrategia de evaluación (por ejemplo, tras cada época)
    evaluation_strategy="epoch",
    # Número de pasos de calentamiento para el planificador de tasa de aprendizaje
    warmup_steps=2,
    # Si se usa precisión de 16 bits (float16)
    fp16=False,
    # Número de pasos entre guardado de puntos de control
    save_steps=500,
    # Límite máximo de puntos de control a conservar
    #save_total_limit=2,
    # Backend de reporte (usar 'none' para desactivar; también puedes reportar en wandb o tensorboard)
    report_to='none'
)

### **Entrenamiento DPO**

El siguiente paso es crear el entrenador usando la clase `DPOTrainer`:


In [ ]:
# Asegura que el token de padding sea el token EOS
tokenizer.pad_token = tokenizer.eos_token

# Crea un entrenador DPO
# Este entrenador manejará el fine-tuning del modelo usando la técnica DPO
trainer = DPOTrainer(
    # Modelo a afinar
    modelo,
    # Modelo de referencia (no se usa en este caso porque LoRA ya está aplicado)
    ref_model=None,
    # Configuración de entrenamiento DPO
    args=training_args,
    # Parámetro beta para la pérdida DPO
    beta=0.1,
    # Conjunto de datos de entrenamiento
    train_dataset=train_dataset,
    # Conjunto de datos de evaluación
    eval_dataset=eval_dataset,
    # Tokenizer del modelo
    tokenizer=tokenizer,
    # Configuración PEFT
    peft_config=peft_config,
    # Longitud máxima del prompt
    max_prompt_length=512,
    # Longitud máxima de la secuencia
    max_length=512,
)

Ten en cuenta que, al usar LoRA en el modelo base, es eficiente dejar `ref_model=None`, de modo que `DPOTrainer` descargará el adaptador para la inferencia de referencia.



#### **Entrenamiento del modelo**


**Ten en cuenta que entrenar el modelo en CPU puede llevar mucho tiempo y puede provocar que el kernel se bloquee por problemas de memoria. Si esto sucede, puedes omitir el entrenamiento cargando el modelo preentrenado que se proporciona en la siguiente sección y continuar desde ahí.**

In [ ]:
# Inicia el proceso de entrenamiento
trainer.train()

Vamos a recuperar y graficar la pérdida de entrenamiento frente a la pérdida de evaluación:


In [ ]:
# Recupera el historial de registros y guárdalo en un DataFrame
log = pd.DataFrame(trainer.state.log_history)
log_t = log[log['loss'].notna()]
log_e = log[log['eval_loss'].notna()]

# Grafica las pérdidas de entrenamiento y evaluación
plt.plot(log_t["epoch"], log_t["loss"], label="train_loss")
plt.plot(log_e["epoch"], log_e["eval_loss"], label="eval_loss")
plt.legend()
plt.show()

In [ ]:
# Carga el modelo DPO entrenado en el último punto de control
dpo_model = AutoModelForCausalLM.from_pretrained('./dpo/checkpoint-250')

#### **Carga del modelo entrenado**


Si encuentras dificultades al ejecutar la celda de entrenamiento por limitaciones de recursos, puedes descargar el modelo ya afinado:


In [ ]:
# Define la URL y el nombre de archivo
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/YIDeT3qihEpWChdXN_RmTg/DPO-tar.gz'
filename = './DPO.tar'

# Descarga el archivo
response = requests.get(url)

# Guarda el archivo localmente
with open(filename, 'wb') as f:
    f.write(response.content)

# Extrae el archivo tar
if tarfile.is_tarfile(filename):
    with tarfile.open(filename, 'r') as tar:
        tar.extractall()
        print("Archivos extraídos:", tar.getnames())
else:
    print("El archivo descargado no es un tar válido.")

Luego, lo cargamos en el modelo para continuar con la inferencia:


In [ ]:
# Carga el modelo DPO entrenado que acabas de descargar
dpo_model = AutoModelForCausalLM.from_pretrained('./DPO')


### **Generación**


In [ ]:
# Carga el tokenizer de GPT-2
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [ ]:
# Fija una semilla para reproducibilidad
set_seed(42)

# Define la configuración de generación para el modelo DPO
# Estos parámetros controlan cómo se genera el texto
generation_config = GenerationConfig(
    # Usa muestreo para producir texto más diverso
    do_sample=True,
    # Parámetro top-k para el muestreo
    top_k=1,
    # Temperatura para controlar la aleatoriedad
    temperature=0.1,
    # Número máximo de tokens nuevos a generar
    max_new_tokens=25,
    # Usa el token de fin de secuencia como token de relleno
    pad_token_id=tokenizer.eos_token_id
)

# Define el texto de entrada para la generación
PROMPT = "Is a higher octane gasoline better for your car?"
# Codifica el prompt con el tokenizer
inputs = tokenizer(PROMPT, return_tensors='pt')

# Genera texto con el modelo DPO
outputs = dpo_model.generate(**inputs, generation_config=generation_config)
# Decodifica y muestra la respuesta
print("Respuesta DPO:\t", tokenizer.decode(outputs[0], skip_special_tokens=True))

# Carga el modelo GPT-2 preentrenado
gpt2_model = AutoModelForCausalLM.from_pretrained('gpt2')
# Genera texto con GPT-2
outputs = gpt2_model.generate(**inputs, generation_config=generation_config)
# Decodifica y muestra la respuesta
print("\nRespuesta GPT-2:\t", tokenizer.decode(outputs[0], skip_special_tokens=True))


Aunque el modelo se entrenó con pocos datos durante solo 5 épocas, se observa que la respuesta generada por el modelo ajustado con DPO es más concisa y directa.


### **Ejercicios** 



#### Ejercicio 1: Preprocesar el conjunto de datos `argilla/ultrafeedback-binarized-preferences-cleaned`

Este conjunto de datos contiene **prompts** generados por usuarios junto con sus respuestas categorizadas como **chosen** o **rejected**, lo que lo hace ideal para entrenar modelos que aprendan las preferencias de los usuarios.



##### Cargar el conjunto de datos desde `argilla/ultrafeedback-binarized-preferences-cleaned`

In [ ]:
#Completa

In [ ]:
dataset['train']

##### Fijar la variable `cnt` en 50 y seleccionar los primeros 50 ejemplos para reducir el volumen de datos

In [ ]:
#Completa

#####  Crea una función `process` que reciba una fila del dataset y elimine columnas no deseadas. 

Columnas a eliminar: `source`, `chosen-rating`, `chosen-model`, `rejected-rating`, `rejected-model`. Luego, aplicar esta función con `map` sobre los datos de entrenamiento.



In [ ]:
#Completa

##### Divide el conjunto en entrenamiento y evaluación

Calcula el tamaño de la partición de entrenamiento como el 80 % del total y el resto (20 %) será para evaluación.


In [ ]:
#Completa

In [ ]:
train_dataset

In [ ]:
train_dataset[0]

####  Ejercicio 2: Inferencia de prompts y comparación con GPT-2


In [ ]:
PROMPT = input()

##### Inicializa el Tokenizer de GPT-2


In [ ]:
#Completa

##### Crea un objeto `generation_config` con los parámetros de generación

* `do_sample=True`      (habilita muestreo para mayor diversidad)
* `top_k=1`             (considera solo el token más probable en cada paso)
* `temperature=0.1`     (controla la aleatoriedad de la generación)
* `max_new_tokens=25`   (número máximo de tokens nuevos)
* `pad_token_id=tokenizer.eos_token_id`  (token de relleno)


In [ ]:
#Completa

##### Crea una función llamada `generate_dpo_response` que reciba un prompt como entrada y genere una respuesta usando el modelo DPO (`dpo_model`).



In [ ]:
#Completa

##### Crea otra función llamada `generate_gpt2_response` que reciba un prompt como entrada y genere una respuesta usando el modelo GPT-2 (`gpt2_model`).


In [ ]:
#Completa

##### Llama a ambas funciones con un prompt y compara las respuestas.



In [ ]:
#Completa